# Energy consumption across Newcastle University campus.

In [ ]:
from datetime import datetime, timedelta, date
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statistics

## The code threw up some SettingWithCopyWarnings that I will fix at somepoint
import warnings
warnings.filterwarnings('ignore')

In [ ]:
metadata = pd.read_pickle('../cache/energy-metadata.pkl')
baseline = pd.read_pickle('../cache/energy-baseline.pkl')
recent = pd.read_pickle('../cache/energy-recent.pkl')

In [ ]:
# Hardcoded list of selected sensors - to be updated for more dynamic selection of locations
sensorID = ['CLZ_E_Tx1',    # Castle Leazes 
           'DEV_E_Main',   # Devonshire 
           'HAN_E_Main',   # Great North Museum 
           'REF_E_Main',   # Kings Road Centre
           'PCY_E_Main',   # Percy Building
           'CSH_E_Main',   # Sports Center 
           'UNS_E_Main',   # Students Union
           'W22_E_Main',   # Windsor Terrace (Law)
           'WOL_E_Main']   # Wolfson Building

In [ ]:
baseline.loc[:,"Weekday"] = baseline.index.weekday
baseline.loc[:,"WeekNumber"] = baseline.index.week

recent.loc[:,"Weekday"] = recent.index.weekday
recent.loc[:,"WeekNumber"] = recent.index.week
recent = recent[recent["WeekNumber"] >= 10]

### Total electricity consumption (kWh) is calculated for each day, and compared to median historical daily total readings from 2017-2019 (calculated from the weeks of the year corresponding to the Covid-19 lockdown).

In [ ]:
# Set up Plots
for sid in sensorID:   
    # Extract data for current Sensor ID
    baselineTemp = baseline[[sid,'Weekday','WeekNumber']]
    recentTemp = recent[[sid,'Weekday','WeekNumber']]
    
    plotDF = pd.DataFrame(columns=["WeekNum",0,1,2,3,4,5,6])
    
    # Extract Baseline Data for Weeks in "Recent" Dataframe
    weeks = recentTemp["WeekNumber"].unique().tolist()
    baselineTemp = baselineTemp[baselineTemp['WeekNumber'].isin(weeks)]
    
    # Get Daily Average SUM Energy Usage for day of week for weeks since Lockdown (2017-2019)
    row = {}
    for wd in range(0,7):
        weekdayBase = baselineTemp[baselineTemp["Weekday"]==wd]
        weekdayBase.loc[:,"Date"] = weekdayBase.index.date
        uniqueDates = weekdayBase["Date"].unique()
        energySumList = []
        for d in uniqueDates:
            dailyBaseDF = weekdayBase[weekdayBase["Date"]==d]
            totalEnergy = dailyBaseDF[sid].sum()
            energySumList.append(totalEnergy)
        aveEnergy = statistics.median(energySumList)
        row[wd] = aveEnergy
    row["WeekNum"] = "Baseline"
    plotDF = plotDF.append(row, ignore_index=True)
    plotDF.rename(index={'0': 'Baseline'}, inplace=True)
    
    # Get SUM Energy Consumption for each Day/Week
    for wk in weeks:
        weekRecent = recentTemp[recentTemp["WeekNumber"]==wk]
        row = {}
        for wd in range(0,7):
            weekdayRecent = weekRecent[weekRecent["Weekday"]==wd]
            totalEnergy = weekdayRecent[sid].sum()
            row[wd] = totalEnergy
        row["WeekNum"] = wk
        plotDF = plotDF.append(row, ignore_index=True)
    
    # Set up Plot Parameters
    wkNums = plotDF["WeekNum"].unique()
    
    x = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

    ind = np.arange(7)
    # Default width - may need updating at later date
    width = 0.05

    numYs = len(wkNums)
    midWeek = (numYs/2) - 0.5
    pos = -(midWeek)
    
    # Define Colours
    colours1 = ('#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c',
                '#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928')
    colours2 = ('#1f78b4','#33a02c','#e31a1c','#ff7f00','#6a3d9a','#b15928')
    col1 = iter(colours1)
    col2 = iter(colours2)
    col3 = iter(colours2)
    
    fig, axs = plt.subplots(figsize=(18,6.5))

    run = 1
    
    # Plotting
    for wkn in wkNums:
        plotRow = plotDF[plotDF["WeekNum"]==wkn]
        plotRow.drop(columns=['WeekNum'], axis=1,inplace=True)
        y = plotRow.values.flatten().tolist()
        if run == 1:
            c = next(col1)
            plt.bar((ind+(width*pos)),y,width,color=c,label="Baseline Values")
        elif run <= 12:
            c = next(col1)
            date = pd.to_datetime('2020' + str(int(wkn) -1 ) + '-1', format='%Y%W-%w')
            monday = date.strftime('%b-%d')
            plt.bar((ind+(width*pos)),y,width,color=c,label="Week of "+monday)
        elif run >= 13 and run <=18:
            c = next(col2)
            date = pd.to_datetime('2020' + str(int(wkn)-1) + '-1', format='%Y%W-%w')
            monday = date.strftime('%b-%d')
            plt.bar((ind+(width*pos)),y,width,hatch="o",fill=False,edgecolor=c,label="Week of "+monday)
        elif run >= 19 and run <=24:
            c = next(col3)
            date = pd.to_datetime('2020' + str(int(wkn)-1) + '-1', format='%Y%W-%w')
            monday = date.strftime('%b-%d')
            ax.bar((ind+(width*pos)),y,width,hatch="x",fill=False,edgecolor=c,label="Week of "+monday)
        pos = pos + 1
        run = run + 1
        
    plt.ylabel("Total Energy Consumption (Electricity (kWh))")

    meta = metadata[metadata["id"] == sid]
    location = meta["description"].values[0]

    plt.title(location, fontsize=14)
    
    plt.legend(bbox_to_anchor=(1.12, 1.01), prop={'size': 9})

    plt.xticks(ind + width / len(plotDF), x)
    
    plt.savefig('../output/energy-usage-' + sid + '.png', bbox_inches='tight')
    
    plt.show()